# Recursive Competitive Equilibria

Contents:

- [Recursive Competitive Equilibria](#Recursive-Competitive-Equilibria)  
  - [Growth Model](#Growth-Model)  
  - [Transition Equations](#Transition-Equations)
  - [Policy Functions](#Policy-Functions)   
  - [Price System](#Price-System)
  - [Simulations](#Simulations)

This lab includes:

(1) An overview of a basic stochastic growth model;

(2) Finding the transition equations of the model using dynamic programming;

(3) Solving for the policy functions of the model using value function iteration;

(4) Solving for the price system of the economy;

(5) Simulating the equilibrium paths of the economy under various initial conditions.

## Growth Model

## Transition Equations 

## Policy Functions

## Price System

## Simulations